In [ ]:
#Code to compare IMB snow depths with latest version of snowmodel
#Run bottom cell first

In [ ]:
Sent_to_Glen=['S3','S4','S13','S20','S22','S23','S26','S27','S28','S30','2013A', '2013B', '2013C', '2013D', '2013E', '2013F', '2013G', '2013H', '2013I', '2014B', '2014C', '2014D', '2014E', '2014I']
track_5_nearest(Sent_to_Glen)
#Plots and stats
line_plots_IMBvsSnodsi(Sent_to_Glen)
buoy_snow_keep, model_snow_keep, buoy_snow5_keep, model_snow5_keep=buoys_stats()
#maps_check('CRREL')
#maps_check('AWI')

In [9]:
plt.clf()



In [7]:
import numpy as np
import glob
import datetime
import scipy.spatial
import os.path
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pyproj import Proj, transform
import datetime
import matplotlib.ticker as tkr
import matplotlib.dates as mdates

def track_5_nearest(buoys_list):
    "Finds 5 nearest Snodsi points to IMB start location and tracks them through time. Saves as .txt file"
    savepath='/Volumes/FREECOMHDD/BUOYS/SNOWMODEL_LAGR/'
    for buoy in range(0, len(buoys_list), 1):
        buoy_name=str(buoys_list[buoy])
        print('Working on buoy '+buoy_name )
        if buoy_name[0]=='S':
            #AWI files have buoys that cover more than one year- the 20* ensures that e.g. S1 and S10 don't get mixed up
            buoy_files=sorted(glob.glob('/Volumes/FREECOMHDD/BUOYS/AWI/SORTED/av_'+buoy_name+'20*.txt'))
        else:
            #CRREL
            buoy_files=sorted(glob.glob('/Volumes/FREECOMHDD/BUOYS/CRREL/SORTED/av_'+buoy_name+'*.txt'))
        
        date=[]
        lons_buoy=[]
        lats_buoy=[]
        snow_buoy=[]

        for file in buoy_files:
                buoy_day=np.loadtxt(file)
                lons_buoy.append(buoy_day[1])
                lats_buoy.append(buoy_day[0])
                snow_buoy.append(buoy_day[2])
                date.append((file[len(file)-12:len(file)-8]+file[len(file)-8:len(file)-6]+file[len(file)-6:len(file)-4]))    
        date= [ int(x) for x in date ]    
        #Read in Glen's file
        file_number='%03d' %(buoy+1)
        model_snow=np.genfromtxt(savepath+file_number+'_snod.dat')
        if len(lons_buoy)>len(model_snow):
            lons_buoy=lons_buoy[0:len(model_snow)]
            lats_buoy=lats_buoy[0:len(model_snow)]
            snow_buoy=snow_buoy[0:len(model_snow)]
            date=date[0:len(model_snow)]
        np.savetxt(savepath+buoy_name+'.txt',np.c_[date, lons_buoy, lats_buoy, snow_buoy, np.ndarray.tolist(model_snow[0:(len(lons_buoy)),4:15])])   
    
    
def line_plots_IMBvsSnodsi(buoy_files):
    "Plots timeseries of snowdepth of buoys and their 5 nearest snodsi points"
    savepath='/Volumes/FREECOMHDD/BUOYS/SNOWMODEL_LAGR/'
    for buoy_name in buoy_files:
            if buoy_name[0:4]=='2014':
                a=0.01
            else:
                a=1
            data=np.genfromtxt(savepath+buoy_name+'.txt')
            if len(data)>0:
                dates=[ str(int(x)) for x in data[:,0] ]  
                dates=[datetime.datetime.strptime(s, "%Y%m%d") for s in dates]
                plt.plot(dates,data[:,3]*a, label='Buoy depth '+buoy_name, color='red') 
                plt.plot(dates,np.mean(data[:,4:9],axis=1)/100, label='Snow Model depth erai', color='green')# color=colours[count] if plotting several for one year
                plt.plot(dates,np.mean(data[:,9:14],axis=1)/100, label='Snow Model depth merra', color='blue')
                plt.xlabel('Day')
                plt.ylabel('snow depth (m)')
                plt.setp(plt.gca().xaxis.get_majorticklabels(),rotation=90)
                #plt.gca().xaxis.set_major_formatter(tkr.FuncFormatter(xfmt))
                if len(data)>10:
                    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=int(len(data[:,3])/10)))
                else:
                    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
                plt.gca().xaxis.set_minor_locator(mdates.DayLocator())
                plt.title('IMB vs Snow Model: buoy '+buoy_name)
                plt.legend(loc='lower right', bbox_to_anchor=(1.39, 0.785))
                plt.savefig(savepath+buoy_name+".png",dpi=100,bbox_inches='tight')
                plt.show()
                plt.clf()

                
                
def buoys_stats():
    "Calculates and saves stats and histograms for all buoys"
    savepath='/Volumes/FREECOMHDD/BUOYS/SNOWMODEL_LAGR/'
    buoys=[]
    mean_bias=[]
    RMSE=[]
    mean_bias_5closest=[]
    RMSE_5closest=[]
    mean_bias.append('mean_bias(m)')
    RMSE.append('RMSE(m)')
    mean_bias_5closest.append('mean_bias_5closest(m)')
    RMSE_5closest.append('RMSE_5closest(m)')
    buoys.append('buoys')

    for buoy_types in ['AWI', 'CRREL']:
        
        if buoy_types=='AWI':
            files=glob.glob(savepath+'S*.txt')
            a,b,c,d,e=4,5,6,7,8
        else:
            files=glob.glob(savepath+'20*.txt')
            a,b,c,d,e=9,10,11,12,13
        
        #For calculating overall stats by buoy types
        model_snow_keep=[]
        model_snow5_keep=[]
        buoy_snow_keep=[]
        buoy_snow5_keep=[]
        
          
        for file in files:
            data=np.genfromtxt(file)
            if data!=[]:
                buoy=file[41:(len(file)-4)]
                f=0.01
                if buoy[0:4]=='2014':
                    g=0.01
                else:
                    g=1
                #Stats for each individual buoy
                model_snow=[]
                model_snow5=[]
                buoy_snow=[]
                buoy_snow5=[]

                buoy_snow.append(data[:,3]*g)
                model_snow.append(data[:,a]*f)
                buoy_snow5.append(data[:,3]*g)
                model_snow5.append(data[:,a]*f)
                buoy_snow5.append(data[:,3]*g)
                model_snow5.append(data[:,b]*f)
                buoy_snow5.append(data[:,3]*g)
                model_snow5.append(data[:,c]*f)                
                buoy_snow5.append(data[:,3]*g)
                model_snow5.append(data[:,d]*f)                    
                buoy_snow5.append(data[:,3]*g)
                model_snow5.append(data[:,e]*f)  
                
                buoy_snow_keep.append(data[:,3]*g)
                model_snow_keep.append(data[:,a]*f)
                buoy_snow5_keep.append(data[:,3]*g)
                model_snow5_keep.append(data[:,a]*f)
                buoy_snow5_keep.append(data[:,3]*g)
                model_snow5_keep.append(data[:,b]*f)
                buoy_snow5_keep.append(data[:,3]*g)
                model_snow5_keep.append(data[:,c]*f)                
                buoy_snow5_keep.append(data[:,3]*g)
                model_snow5_keep.append(data[:,d]*f)                    
                buoy_snow5_keep.append(data[:,3]*g)
                model_snow5_keep.append(data[:,e]*f) 

                mean_bias.append(np.nanmean(np.array(model_snow)-np.array(buoy_snow)))
                RMSE.append(np.nanmean((np.array(model_snow)-np.array(buoy_snow))**2.0)**(1/2.0))
                mean_bias_5closest.append(np.nanmean(np.array(np.ravel(model_snow5))-np.array(np.ravel(buoy_snow5))))
                RMSE_5closest.append(np.nanmean((np.array(np.ravel(model_snow5))-np.array(np.ravel(buoy_snow5)))**2.0)**(1/2.0))
                buoys.append(buoy)

                model_snow_hist=np.array((model_snow5))
                buoy_snow_hist=np.array(buoy_snow)

                bins = np.linspace(0, 1, 21)
                plt.hist(model_snow_hist[~np.isnan(model_snow_hist)], bins, alpha=0.5, label='modelled snow')
                plt.hist(buoy_snow_hist[~np.isnan(buoy_snow_hist)], bins, alpha=0.5, label='buoy snow')
                plt.legend(loc='upper right')
                plt.title('Modelled and buoys snow depths for'+buoy)
                plt.xlabel('snow depth (m)')
                plt.savefig(savepath+'Hist_'+str(buoy)+'.png')
                plt.clf()
        
        mean_bias.append(np.nanmean(np.concatenate(model_snow_keep)-np.concatenate(buoy_snow_keep)))
        RMSE.append(np.nanmean((np.concatenate(model_snow_keep)-np.concatenate(buoy_snow_keep))**2.0)**(1/2.0))
        mean_bias_5closest.append(np.nanmean(np.concatenate(np.ravel(model_snow5_keep))-np.concatenate(np.ravel(buoy_snow5_keep))))
        RMSE_5closest.append(np.nanmean((np.concatenate(np.ravel(model_snow5_keep))-np.concatenate(np.ravel(buoy_snow5_keep)))**2.0)**(1/2.0))
        buoys.append(buoy_types+'_all')

        model_snow_hist=np.concatenate(model_snow5_keep)
        buoy_snow_hist=np.concatenate(buoy_snow_keep)

        bins = np.linspace(0, 1, 21)
        plt.hist(model_snow_hist[~np.isnan(model_snow_hist)], bins, alpha=0.5, label='modelled snow')
        plt.hist(buoy_snow_hist[~np.isnan(buoy_snow_hist)], bins, alpha=0.5, label='buoy snow')
        plt.legend(loc='upper right')
        plt.title('Modelled and buoys snow depths for all '+buoy_types+' buoys')
        plt.xlabel('snow depth (m)')
        plt.savefig(savepath+'Hist_'+buoy_types+'_all.png')
        plt.clf()
        

    np.savetxt(savepath+'_stats.txt',np.c_[buoys, mean_bias, RMSE, mean_bias_5closest, RMSE_5closest],fmt='%s')
    return (buoy_snow_keep, model_snow_keep, buoy_snow5_keep, model_snow5_keep)
    
def maps_check(buoys):
    "Plots path of IMB and 5 closest Snodsi points"
    savepath='/Volumes/FREECOMHDD/BUOYS/SNOWMODEL_LAGR/'
    files=glob.glob(savepath+'*'+buoys+'*.txt')
    for file in files:
        data=np.genfromtxt(file)
        if data!=[]:
            buoy_lons=[]
            buoy_lats=[]
            model_lons1=[]
            model_lats1=[]
            model_lons2=[]
            model_lats2=[]
            model_lons3=[]
            model_lats3=[]
            model_lons4=[]
            model_lats4=[]
            model_lons5=[]
            model_lats5=[]
            buoy_lons.append(data[:,1])
            buoy_lats.append(data[:,2])
            model_lons1.append(data[:,4])
            model_lats1.append(data[:,5])
            model_lons2.append(data[:,7])
            model_lats2.append(data[:,8])
            model_lons3.append(data[:,10])
            model_lats3.append(data[:,11])
            model_lons4.append(data[:,13])
            model_lats4.append(data[:,14])
            model_lons5.append(data[:,16])
            model_lats5.append(data[:,17])
            m = Basemap(projection='npstere',boundinglat=50,lon_0=0,resolution='l')
            m.drawparallels(np.arange(60,90,10), linewidth = 0.25, linestyle='solid', zorder=8)
            m.drawmeridians(np.arange(0.,360.,30.), linewidth = 0.25, zorder=8)
            m.drawcoastlines(linewidth=0.5)
            x2,y2=m(np.ravel(model_lons1),np.ravel(model_lats1))
            m.scatter(x2,y2, s=1, marker='o',color='k')
            x2,y2=m(np.ravel(model_lons2),np.ravel(model_lats2))
            m.scatter(x2,y2, s=1, marker='o',color='b')
            x2,y2=m(np.ravel(model_lons3),np.ravel(model_lats3))
            m.scatter(x2,y2, s=1, marker='o',color='g')
            x2,y2=m(np.ravel(model_lons4),np.ravel(model_lats4))
            m.scatter(x2,y2, s=1, marker='o',color='y')
            x2,y2=m(np.ravel(model_lons5),np.ravel(model_lats5))
            m.scatter(x2,y2, s=1, marker='o',color='c')
            x1,y1=m(np.ravel(buoy_lons),np.ravel(buoy_lats))
            m.scatter(x1,y1, s=1, marker='x',color='r')
            m.fillcontinents(color='grey')
            plt.title(str(file[46:(len(file)-4)]))
            plt.savefig(savepath+str(file[46:(len(file)-4)])+"_map.png",dpi=100,bbox_inches='tight')
            plt.show() 
            plt.clf()